# Підсумковий урок: розбір, доопрацювання та деплой навчального проєкту

Проєкт: **car_service_api_and_ui**
(Backend: Django + DRF, Frontend/UI: Django templates або API-клієнт)

---

## Мета підсумкового уроку

На цьому уроці ми:

* розберемо структуру навчального проєкту;
* додамо новий функціонал **“робота по машині”**;
* реалізуємо поля **"вартість"** та **"оплачено"**;
* пройдемо повний цикл: **код → PR → деплой → тестування**;
* закріпимо реальний production-пайплайн.

---

## 1. Огляд проєкту

### Типова структура

```
car_service/
 ├── api/            # DRF (models, serializers, views)
 ├── ui/             # templates / views
 ├── car_service/    # settings, urls
 ├── manage.py
 └── requirements.txt
```

### Ключові сутності домену

* Car (авто)
* Client (клієнт)
* Service / Work (робота по машині)

📌 Наша задача — **розширити модель Work**.

---

## 2. Постановка задачі (business requirements)

Для **роботи по машині** потрібно:

1. Додати поле **cost** (вартість)
2. Додати поле **is_paid** (чи оплачено)
3. Відображати ці дані:

   * в API
   * в UI
4. Забезпечити можливість:

   * створення
   * редагування
   * перегляду

---

## 3. Оновлення моделі

### models.py

```python
class Work(models.Model):
    car = models.ForeignKey(Car, on_delete=models.CASCADE)
    description = models.TextField()
    cost = models.DecimalField(max_digits=10, decimal_places=2)
    is_paid = models.BooleanField(default=False)
    created_at = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return f"{self.car} - {self.cost}"
```

📌 `DecimalField` — стандарт для грошей

---

## 4. Міграції

```bash
python manage.py makemigrations
python manage.py migrate
```

⚠️ Типова помилка студентів — забути `migrate`

---

## 5. Оновлення DRF Serializer

### serializers.py

```python
class WorkSerializer(serializers.ModelSerializer):
    class Meta:
        model = Work
        fields = ['id', 'car', 'description', 'cost', 'is_paid', 'created_at']
```

📌 Без цього API не побачить нові поля

---

## 6. Оновлення API Views

### Перевіряємо:

* GET → повертає cost та is_paid
* POST → приймає cost та is_paid
* PATCH → дозволяє змінювати is_paid

```json
{
  "car": 1,
  "description": "Oil change",
  "cost": "1200.00",
  "is_paid": true
}
```

---

## 7. Оновлення UI (Templates / Forms)

### forms.py

```python
class WorkForm(forms.ModelForm):
    class Meta:
        model = Work
        fields = ['car', 'description', 'cost', 'is_paid']
```

### template (checkbox + cost)

```html
<label>Cost</label>
<input type="number" step="0.01" name="cost">

<label>Paid</label>
<input type="checkbox" name="is_paid">
```

📌 Checkbox → BooleanField

---

## 8. Бізнес-логіка: неоплачена робота

### Приклад фільтрації

```python
unpaid_works = Work.objects.filter(is_paid=False)
```

Можливі розширення:

* підрахунок загальної заборгованості
* звіт по клієнту

---

## 9. Pull Request (PR)

### Правильний workflow

1. Створюємо feature-гілку

```bash
git checkout -b feature/work-cost-paid
```

2. Коміти

```bash
git add .
git commit -m "Add cost and paid flag to work"
```

3. Push

```bash
git push origin feature/work-cost-paid
```

4. Створюємо **Pull Request**

### Що перевіряємо в PR:

* міграції
* серіалізатори
* backward compatibility
* відсутність DEBUG / SECRET_KEY

---

## 10. Деплой проєкту

### Production checklist

* DEBUG = False
* ALLOWED_HOSTS
* collectstatic
* migrate

```bash
python manage.py collectstatic
python manage.py migrate
```

### Перезапуск сервісів

```bash
sudo systemctl restart gunicorn
sudo systemctl reload nginx
```

---

## 11. Тестування після деплою

### API testing (Postman / curl)

* GET /api/works/
* POST /api/works/
* PATCH /api/works/{id}/

Перевіряємо:

* статус-коди (200 / 201)
* типи даних
* Boolean is_paid

---

### UI testing

* створення роботи
* введення вартості
* встановлення прапорця "оплачено"
* збереження

---

## 12. QA-фокус: що тестувати

* негативні кейси (cost < 0)
* відсутні поля
* права доступу
* регресію старого функціоналу

---

## 13. Типові помилки студентів

❌ Забули serializer

❌ BooleanField + checkbox обробляється неправильно

❌ Не зробили migrate на сервері

❌ Тестували тільки локально

---

## 14. Підсумок уроку

На цьому уроці ми:

* доопрацювали бізнес-модель
* пройшли повний шлях feature development
* зробили PR
* задеплоїли в production
* протестували як API, так і UI

📌 Це **максимально наближено до реальної роботи**

---

## Фінальне завдання

1. Додати валідацію `cost > 0`
2. Додати фільтр: тільки неоплачені роботи
3. Написати короткий README для feature

---

🎓 Курс завершено. Ви маєте **повний Django-пайплайн: від моделей до production**
